In [1]:
import nbimporter
import moisture_conversion as mc

Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb


# 温度・水分の状態を保持するモジュール

水蒸気の状態を保持するクラスを作成する。  
水蒸気の状態を表す方法として以下の4つの方法が挙げられる。  

- $RH$ 相対湿度(0.0~1.0)
- $\mu$ 水分化学ポテンシャル, J kg<sup>-1<sup>
- $P_v$ 水蒸気圧, Pa
- $X$ 絶対湿度, kg kg<sup>-1</sup>DA  

ここでは、相対湿度・水分化学ポテンシャル・水蒸気圧　の3つの方法で状態を保持するクラスを用意し、変数を作る時に使い分けることにする。3つのクラスのどれを使用しても計算結果にほとんど差は見られないが、内部的にコンバートする処理をなるべく少なくし計算を高速化させる意味でも、プログラム内で最も良く使う水分状態量と同じクラスを使用することが望ましい。

## クラス定義

### 1) 水分化学ポテンシャルベース  

各コマンドの用途と名称  

class = MiuBasedMoisture( temp, miu )　とする。    

- class.temp　：　温度を出力する。  
- class.miu　：　水分化学ポテンシャルを出力する。  
- class.rh　：　相対湿度を計算する。  
- class.pv　：　水蒸気圧を計算する。  


- class.set_temp　：　クラスに温度を再設定する。  
- class.set_miu　：　クラスに水分化学ポテンシャルを再設定する。  
- class.set_rh　：　クラスに相対湿度から算出した水分化学ポテンシャルを再設定する。  
- class.set_pv　：　クラスに水蒸気圧から算出した水分化学ポテンシャルを再設定する。

In [2]:
class MiuBasedMoisture():
    
    def __init__(self, temp, miu):
        self._temp = temp
        self._miu = miu
    
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp
    
    temp = property(get_temp, set_temp)

    # miu
    
    def set_miu(self, miu):
        self._miu = miu
        
    def get_miu(self):
        return self._miu
    
    miu = property(get_miu, set_miu)
    
    # pv
    
    def set_pv(self, pv):
        self._miu = mc.convertPv2Miu(self._temp, pv)
    
    def get_pv(self):
        return mc.convertMiu2Pv(self._temp, self._miu)
    
    pv = property(get_pv, set_pv)
    
    # RH
    
    def set_RH(self, rh):
        self._miu = mc.convertRH2Miu(self._temp, rh)
        
    def get_RH(self):
        return mc.convertMiu2RH(self._temp, self._miu)    
    
    rh = property(get_RH, set_RH)

### 2) 相対湿度ベース

In [3]:
class RHBasedMoisture():
    
    def __init__(self, temp, rh):
        self._temp = temp
        self._rh = rh
    
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp

    temp = property(get_temp, set_temp)

    # miu
    
    def set_miu(self, miu):
        self._rh = mc.convertMiu2RH(self._temp, miu)
        
    def get_miu(self):
        return mc.convertRH2Miu(self._temp, self._rh)
    
    miu = property(get_miu, set_miu)

    # pv
    
    def set_pv(self, pv):
        self._rh = mc.convertPv2RH(self._temp, pv)
    
    def get_pv(self):
        return mc.convertRH2Pv(self._temp, self._rh)
    
    pv = property(get_pv, set_pv)

    # RH
    
    def set_RH(self, rh):
        self._rh = rh
        
    def get_RH(self):
        return self._rh

    rh = property(get_RH, set_RH)

### 3) 水蒸気圧ベース

In [4]:
class PVBasedMoisture():
    
    def __init__(self, temp, pv):
        self._temp = temp
        self._pv = pv
    
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp

    temp = property(get_temp, set_temp)

    # miu
    
    def set_miu(self, miu):
        self._pv = mc.convertMiu2Pv(self._temp, miu)
        
    def get_miu(self):
        return mc.convertPv2Miu(self._temp, self._pv)
    
    miu = property(get_miu, set_miu)

    # pv
    
    def set_pv(self, pv):
        self._pv = pv
    
    def get_pv(self):
        return self._pv
    
    pv = property(get_pv, set_pv)

    # RH
    
    def set_RH(self, rh):
        self._pv = mc.convertRH2Pv(self._temp, rh)
        
    def get_RH(self):
        return mc.convertPv2RH(self._temp, self._pv)
    
    rh = property(get_RH, set_RH)

## 使い方

変数の宣言  
例として、

- 絶対温度 300 K
- 相対湿度 0.5(50%)  

とする。

In [5]:
m1 = RHBasedMoisture(temp=300.0, rh=0.5)

温度の取得

In [6]:
m1.temp

300.0

相対湿度の取得

In [7]:
m1.rh

0.5

水分化学ポテンシャルの取得

In [8]:
m1.miu

-95984.87933286809

水蒸気圧の取得

In [9]:
m1.pv

1768.0065133034425

同様に、水分化学ポテンシャルベースで以下の状態量を宣言する。

- 絶対温度 300.0 K
- 水分化学ポテンシャル -95984.87933286809

なお、上記の値は、300K, RH = 50% の状態と同じである。 

In [10]:
m2 = MiuBasedMoisture(temp=300.0, miu=-95984.87933286809)

相対湿度の取得

In [11]:
m2.rh

0.5000000000000001

水分化学ポテンシャルの取得

In [12]:
m2.miu

-95984.87933286809

水蒸気圧の取得

In [13]:
m2.pv

1768.006513303443

同様に、水蒸気圧ベースで以下の状態量を宣言する。

- 絶対温度 300.0 K
- 水蒸気圧 1768.006513303443 Pa

なお、上記の値は、300K, RH = 50% の状態と同じである。 

In [14]:
m3 = PVBasedMoisture(temp=300.0, pv=1768.006513303443)

相対湿度の取得

In [15]:
m3.rh

0.5000000000000001

水分化学ポテンシャルの取得

In [16]:
m3.miu

-95984.87933286806

水蒸気圧の取得

In [17]:
m3.pv

1768.006513303443

どの方法で宣言しても、計算結果は同じになる。ただし、それぞれ、内部的に水分状態をコンバートしているか否かはクラスごとに異なる。

また、このクラスはさまざまな関数で利用されることが想定される。例えば以下のように水蒸気圧差を出す関数があったとしよう。

In [18]:
def get_pv_dif(m1, m2):
    return m1.pv - m2.pv

3つのクラスの全てで同じ名前の関数を用意しているため、どのタイプのクラスを引数にしようが、変数宣言後は内部的に何の値で水分状態を保持しているのかを全く意識せずにコーディング可能である。

In [19]:
m_a1 = RHBasedMoisture(temp=300.0, rh=0.5)
m_a2 = MiuBasedMoisture(temp=300.0, miu=-95984.87933286809)
m_a3 = PVBasedMoisture(temp=300.0, pv=1768.006513303443)

In [20]:
m_b1 = RHBasedMoisture(temp=300.0, rh=0.3)
m_b2 = MiuBasedMoisture(temp=300.0, miu=-166722.43296138686)
m_b3 = PVBasedMoisture(temp=300.0, pv=1060.8039079820655)

変数 m_a1, m_a2, m_a3 はどれも表現方法が異なるだけで、同じ状態である。  
m_b1, m_b2, m_b3 も同様である。
これらを組み合わせた結果をどれも同じである。

In [21]:
get_pv_dif(m_a1, m_b1)

707.202605321377

In [22]:
get_pv_dif(m_a2, m_b2)

707.2026053213774

In [23]:
get_pv_dif(m_a3, m_b3)

707.2026053213774

異なるクラスをミックスして使用することも可能である。

In [24]:
get_pv_dif(m_a1, m_b2)

707.202605321377

In [25]:
get_pv_dif(m_a1, m_b3)

707.202605321377

In [26]:
get_pv_dif(m_a2, m_b1)

707.2026053213774

In [27]:
get_pv_dif(m_a2, m_b3)

707.2026053213774

In [28]:
get_pv_dif(m_a3, m_b1)

707.2026053213774

In [29]:
get_pv_dif(m_a3, m_b2)

707.2026053213774

## Description

水蒸気の状態量は以下の4つの方法で表現が可能である。  

- $RH$ 相対湿度(0.0~1.0)
- $\mu$ 水分化学ポテンシャル, J kg<sup>-1<sup>
- $P_v$ 水蒸気圧, Pa
- $X$ 絶対湿度, kg kg<sup>-1</sup>DA  
    
それぞれの物理的意味は、鉾井修一ら；エース建築環境工学Ⅱより  
### $RH$：相対湿度
空中の水蒸気圧fの、その温度における飽和水蒸気圧fsに対する割合を百分率で表したものであり、  

$RH = \frac{f}{f_s}×100$  
    
### $P_v$：水蒸気圧
空気中に含まれている水蒸気の分圧を水蒸気圧（vapour pressure）f[Pa]という。  
空間1m<sup>3</sup>中に含みうる水蒸気の量は、温度に応じて限界あり、その温度における限界量を含んだときの水蒸気圧を飽和水蒸気圧という。  
これをfs[Pa]とすると、fsは温度[T]とともに大きくなるが、その関係式についてはゴフ―グラッチ（Goff-Gratch）、ゾンタック（Sonntag）、ウェクスラーハイランド（Wexler-Hyland）などが有名であろう。
    
- Goff-Gratchの飽和水蒸気圧式：  
    $log(f_s) = -7.90298 (373.16/T-1) + 5.02808 log(373.16/T) - 1.3816 10^{-7}(10^{11.344 (1-T/373.16)} -1) + 8.1328 10^{-3} (10{-3.49149(373.16/T-1)} -1) + log(1013.246)$ 
  
  
- Sonntagの飽和水蒸気圧式：  
    $ln(f_s) = -6096.9385 / T + 16.635794 - 2.711193 10^{-2} T + 1.673952 10^{-5} T^{2} + 2.433502 ln(T)$  
  
  
- Wexler-Hylandの飽和水蒸気圧式：  
    $ln(f_s) = -0.58002206104 / T + 0.13914993 10^1 - 0.48640239 10^{-1} T + 0.41764768 10^{-4} T^{2} - 0.14452093 10^{-7} T^{3} + 0.65459673 10^{1}ln(T) $

### $X$ ：絶対湿度
湿り空気中の乾燥空気1kg'当りに含まれる水分量がX[kg]のとき、この空気の絶対湿度（absolute humidity）はX[kg/kg']であるという。水分が水蒸気のみである場合には、絶対湿度と水蒸気圧の関係は、  

$X = 0.622×f/(F-f)$

ここでFは大気圧である。
絶対湿度は空気が除湿または吸湿されない限り、温度が変わっても変化しない。  

### $\mu$：水分化学ポテンシャル  
化学ポテンシャルはJ.W.Gibbsにより提唱された状態変数の一つであり、1molあたりのギブスの自由エネルギーと定義される。  
ここで一般に1kg当たりのエネルギー量とした方が簡便であることから、水蒸気の化学ポテンシャルは飽和時の水蒸気圧を基準として以下のように定められる。 

$\mu = R_v T log(RH)$ [J/kg] 

Rvは水蒸気の気体定数である。